In [52]:
import torch
import numpy as np
%load_ext autoreload
%autoreload 2
from utils import load_data,default_loader,trainset
from torch.utils.data import Dataset, DataLoader

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
datapath = "/home/tete/data/Youtube"

In [7]:
data,paths,labels = load_data(datapath)

In [10]:
available_field = ['url','videoname','locs','frameids','label_names','crop','scale','origRes','isYouTubeSubset']

In [11]:
for str in available_field:
    print("{}:{}".format(str,data[0][str]))

url:https://www.youtube.com/watch?v=-osma2n86oA
videoname:-osma2n86oA
locs:[[[195.49831989 207.3249328  229.00705645 ... 234.07560484 204.50907258
   225.06485215]
  [158.04737903 135.23891129 181.98219086 ... 204.50907258 167.05813172
   177.47681452]
  [278.56619624 274.34240591 213.80141129 ... 303.34576613 270.68178763
   257.72883065]
  ...
  [277.72143817 273.49764785 258.01041667 ... 299.12197581 305.88004032
   277.15826613]
  [178.6031586  181.1374328  190.99294355 ... 209.85920699 180.01108871
   185.64280914]
  [254.63138441 254.34979839 250.40759409 ... 278.00302419 252.37869624
   259.41834677]]

 [[ 81.47143817  72.74227151  76.96606183 ...  81.18985215  77.52923387
    84.56888441]
  [122.30141129  91.60853495  89.07426075 ...  84.85047043  89.6374328
   180.87130376]
  [235.49899194 235.49899194 108.78528226 ... 232.4015457  152.14952957
   191.00840054]
  ...
  [192.41633065 187.62936828 159.75235215 ... 191.8531586  163.69455645
   199.17439516]
  [134.69119624 123.99

In [12]:
len(data[0]['locs'])

2

In [37]:
len(labels),len(labels[0]),len(labels[0][0]),len(paths)

(5000, 7, 2, 5000)

In [62]:
class ExperimentParams():
    def __init__(self):
        self.data_dir = '/home/tete/'
        self.num_classes = 10
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.batch_size = 256
        self.num_epochs = 20
        self.num_workers = 4
        self.lr = 1e-2
        
        self.drop_prob1 = 0.2
        self.drop_prob2 = 0.5

args = ExperimentParams()

In [68]:
train_data  = trainset(paths,labels)
trainloader = DataLoader(train_data, batch_size=4,shuffle=True)

In [76]:
trainloader.dataset[1][0].shape

torch.Size([3, 236, 419])

In [78]:
count = 0
for data in trainloader.dataset:
    if count == 1:
        inputs_try,labels_try = data
    count +=1
print(count)

5001


In [69]:
for batch_idx, (data, target) in enumerate(trainloader):
    print(batch_idx)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 464 and 216 in dimension 2 at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorMoreMath.cpp:1333

In [67]:
class StupidNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32 ,kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64 ,kernel_size=5)
        self.fc1 = nn.Linear(in_features=16*64, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=2)

    def forward(self, x, debug=False):
        x1 = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2, stride=2)
        x2 = F.max_pool2d(F.relu(self.conv2(x1)), kernel_size=2, stride=2)
        x3 = x2.view([-1,4*4*64])
        if debug == True:
            print(f'input: {x.size()}')
            print(self.conv1(x).shape)
            print(f'x1: {x1.size()}')
            print(f'x2: {x2.size()}')  
            print(f'x3: {x3.size()}')
        x4 = F.relu(self.fc1(x3))
        x5 = F.relu(self.fc2(x4))
        output = F.relu(self.fc3(x5))              
             
        return output

In [61]:
def train_classif_epoch(train_loader, model, loss_fn, optimizer, args, log_interval=100):
    model.train()
    losses = []
    total_loss, total_corrects, num_samples = 0, 0, 0
    corrects = 0.    
    for batch_idx, (data, target) in enumerate(train_loader):
        num_samples += data.size(0)
        
        data, target = data.to(args.device), target.to(args.device)
        
        optimizer.zero_grad()
        outputs = model(data)

        loss = loss_fn(outputs, target)
        losses.append(loss.data.item())

        _,preds = torch.max(outputs.data,1)
        corrects += torch.sum(preds == target.data).cpu()

        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tAccuracy: {}'.format(
                batch_idx * len(data[0]), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses), float(total_corrects)/num_samples))           
            
            total_loss += np.sum(losses)
            total_corrects += corrects
            losses, corrects = [], 0
            
    accuracy = total_corrects.item()/num_samples
    return total_loss/(batch_idx + 1), accuracy

def test_classif_epoch(test_loader, model, loss_fn, args, log_interval=100):
    with torch.no_grad():
        model.eval()
        losses, corrects = [], 0
        num_samples = 0
        corrects = 0.
        for batch_idx, (data, target) in enumerate(test_loader):

            num_samples += data.size(0)
            data, target = data.to(args.device), target.to(args.device)

            outputs = model(data)

            loss = loss_fn(outputs, target)
            losses.append(loss.data.item())

            _,preds = torch.max(outputs.data,1)
            corrects += torch.sum(preds == target.data).cpu()

        accuracy = corrects.item()/num_samples
        return np.sum(losses)/(batch_idx + 1), accuracy